In [3]:
import pandas as pd
import numpy as np
from pandas import Series


In [4]:
import random

In [5]:
import pyspark_csv as pycsv
from pyspark import SparkContext 
sc = SparkContext.getOrCreate()
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Distancias

In [44]:
def get_minkowsky_distance(p):

    def minkowsky_distance(v1, v2):
        if(len(v1) != len(v2)):
            raise ValueError("The vectors do not have the same dimention")
        s = 0
        for i in range(len(v1)):
            s += abs((v1[i]-v2[i])**p)
        if(p == 0): return s
        return round((s)**(1.0/p),2)
    return minkowsky_distance

hamming = get_minkowsky_distance(0)
manhattan = get_minkowsky_distance(1)
euclidean = get_minkowsky_distance(2)

In [86]:
df= pd.read_csv('../tp1/data_filled_ready_to_train.csv',encoding='UTF-8')

In [87]:
def split_train_test_sets(df, porcentage):
    indices = random.sample(df.index, (int)(df.shape[0]*porcentage))
    return df.drop(indices),df.iloc[indices]




In [88]:
train, test = split_train_test_sets(df, 0.33)
test = test.loc[:,['created_on','property_type','lat','lon', 'place_name',
       'state_name', 'surface_total_in_m2', 'surface_covered_in_m2',
       'description', 'floor', 'rooms', 'expenses']]

In [90]:
#esto tarda
X_train_rdd = sqlContext.createDataFrame(train).rdd
X_test_rdd = sqlContext.createDataFrame(test).rdd


In [99]:
X_train_rdd.take(1)

[Row(created_on=94953600.0, property_type=1, lat=-34.6037137392, lon=-58.3815808896, place_name=430, state_name=3, surface_total_in_m2=52.0, surface_covered_in_m2=52.0, description=20, floor=1.0, rooms=1.0, expenses=10.0, price_aprox_usd=125000.0, price_usd_per_m2=2403.84615385)]

In [ ]:
def get_X()

def get_nearest_neighbors(x, X_train_rdd):
    X_train_rdd.map(lambda neighbor: (neighbor,euclidean(x,neighbor))).sortBy(lambda (neighbor,distance): distance)

    


In [ ]:
nn = X_test_rdd.map(lambda x: get_nearest_neighbors(x,X_train_rdd))

# USANDO LA LIBRERIA

In [229]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression


In [392]:
(max(df.price_usd_per_m2)-min(df.price_usd_per_m2))/70

54.771294695100003

In [231]:
min(df.price_usd_per_m2)

226.923076923

In [232]:
int((3000.89-226)/4)

693

In [357]:
df= pd.read_csv('../tp1/data_filled_ready_to_train.csv',encoding='UTF-8')
df['bin'] = df.price_usd_per_m2.map(lambda value: int((value-226)/50) )


In [393]:
X = df.loc[:,['created_on','property_type','lat','lon', 'place_name',
       'state_name', 'surface_total_in_m2', 'surface_covered_in_m2',
       'description', 'floor', 'rooms', 'expenses']]
y = df.bin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)


In [394]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train) 
predictions = neigh.predict(X_test)


In [395]:
# el error cuadratico medio
print("Mean squared error: %.2f" % mean_squared_error(y_test, predictions))

Mean squared error: 272.07


In [396]:
porcent =100*abs((predictions-y_test))/y_test
porcent.mean()

inf

In [397]:
df_to_predict= pd.read_csv('../tp1/data_filled_ready_to_predict.csv',encoding='UTF-8')

In [398]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X.loc[:,['lat','lon']],  df['bin']) 
predictions = neigh.predict(df_to_predict.loc[:,['lat','lon']])

In [399]:
df_to_predict['bin'] = predictions
properati_no_price= pd.read_csv('../tp1/properati_dataset_testing_noprice.csv',encoding='UTF-8')
df_to_predict['id']=properati_no_price.id

In [424]:
def linear_regression(df_to_predict, X_train, y_train):
    regr = LinearRegression()
    regr.fit(X_train, y_train)
    y_price_usd_predicted = regr.predict(df_to_predict)
    return y_price_usd_predicted

In [426]:


r = pd.DataFrame()
for b in bins:
    print b
    df_bin = df_to_predict.loc[df_to_predict.bin == b,:]
    x = df.loc[df.bin == b,:]
    df_bin['price_usd']  = linear_regression(df_bin.loc[:,columns], x.loc[:,columns], x.price_aprox_usd)
    
    
    ready_df= df_bin.loc[:,['id','price_usd']]
    r = pd.concat([r,ready_df])


3
43
17
26


/home/jaz/fiuba/orga-de-datos-7506/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


35
14
47
74
32
8
56
54
39
37
46
42
36
59
22
28
10
41
38
21
62
31
70
71
6
29
25
30
15
57
20
53
40
5
49
48
45
58
50
11
12
23
52
18
27
24
44
60
13
9
64
19
75
51
34
65
55
63
2
0
1
7
4
16
76
67
33
72
73
69
68
66
61


In [430]:
r.price_usd = r.price_usd.map(lambda v: int(v))

In [431]:
prediccion1= pd.read_csv('../tp1/prediccion1.csv',encoding='UTF-8')

In [435]:
r.to_csv("prediccion2.csv",encoding='utf-8',index=False)

In [436]:
r.shape

(14166, 2)